In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

df = pd.read_csv('../../data/coupang_filtering.csv', encoding='utf8')

In [2]:
# def category_name(product):
#     if ('닭' in product)or('백숙' in product): name = '닭'
#     elif '구이' in product : name = '구이'
#     elif ('전골' in product)or('나베' in product)or('스키야키' in product)or('샤브샤브' in product): name = '전골'
#     elif ('해물탕' in product)or('매운탕' in product) : name = '매운탕·해물탕'
#     elif ('알탕' in product)or('곤이탕' in product)or('알이 맛있탕' in product) : name = '알탕'
#     elif ('찌개' in product) or ('짜글이' in product) : name = '찌개/짜글이'
#     elif '족발' in product : name = '족발'
#     elif '순대' in product : name = '순대'
#     elif ('파스타' in product) or ('라자냐' in product) : name = '파스타'
#     elif ('국수' in product)or('수제비' in product) : name = '국수·칼국수·수제비'

#     elif ('국밥' in product)  : name = '국밥'
#     elif '볶음' in product : name = '볶음'
#     elif '덮밥' in product : name = '덮밥'
    
#     elif '리조또' in product : name = '리조또'
#     elif '샌드위치' in product : name = '샌드위치'
#     elif ('떡볶이' in product) or ('떡볶잉' in product) : name = '떡볶이'
 
#     elif '마라' in product : name = '마라'
#     elif '마파두부' in product : name = '마파두부'
#     elif ('고추 잡채' in product)or('고추잡채' in product) : name = '고추잡채'
#     elif '스테이크' in product : name = '스테이크'
#     elif '감바스' in product : name = '감바스'
#     elif '무침' in product : name = '무침'
    
#     elif ('해물' in product)or('씨푸드' in product)or('씨키트' in product)or('꽃게' in product)or('새우' in product): name = '해산물요리'
#     elif ('쉬림프' in product)or('연어' in product)or('조개' in product)or('문어' in product)or('오징어' in product): name = '해산물요리'

#     elif ('잡채' in product)or('비빔밥' in product) : name='한식'
#     elif ('부침개' in product)or('조림' in product): name = '한식'

#     elif '오리' in product : name = '오리'
#     elif '불고기' in product : name = '불고기'
#     elif ('커리' in product) or ('카레' in product) : name = '카레'
#     elif '찜' in product : name = '찜'

#     elif '쌈' in product : name = '쌈'
    
#     elif ('국' in product) or ('육개장' in product) or ('파개장' in product) : name = '국'
#     elif ('면' in product)or('소바' in product)or('짜장' in product)or('누들' in product) : name = '면'
#     elif ('우동' in product)or('라면' in product) : name = '면'
#     elif ('탕' in product)or('짬뽕탕' in product) : name = '탕'

#     else: name='기타'
        
#     return name

In [3]:
def category_name(product):
    if ('닭' in product) or ('백숙' in product) : name = '닭'
    elif '구이' in product : name = '구이'
    elif ('전골' in product)or('나베' in product)or('스키야키' in product) or ('샤브샤브' in product): name = '전골'
    elif '족발' in product : name = '족발'
    elif '국수' in product : name = '국수'
    elif ('해물탕' in product) or ('연포탕' in product) or ('해신탕' in product) or ('꽃게탕' in product) or ('해물 누룽지탕' in product) or ('조개탕' in product) or ('바지락탕' in product): name = '해물탕'
    elif ('해물뚝배기' in product) or ('문어오리탕' in product) or ('홍합탕' in product) : name = '해물탕'
    elif ('매운탕' in product) or ('대구탕' in product) or ('동태탕' in product) or ('지리탕' in product)  or ('맑은탕' in product) or ('맑음탕' in product): name = '생선탕'
    elif ('알탕' in product) or ('알이 맛있탕' in product) or ('곤이탕' in product): name = '알탕'

    elif ('찌개' in product) or ('짜글이' in product) or ('짬뽕 순두부' in product): name = '찌개/짜글이'

    elif ('면' in product)or('소바' in product)or('짜장' in product)or('누들' in product)or('짬뽕' in product) : name = '면'
    elif ('우동' in product)or('라면' in product) : name = '면'
    elif '스테이크' in product : name = '스테이크'

    elif ('파스타' in product) or ('라자냐' in product) : name = '파스타'

    elif '리조또' in product : name = '리조또' 
        
    elif '마라탕' in product : name = '마라탕'
    elif '마파두부' in product : name = '마파두부'

    elif ('떡볶이' in product) or ('떡볶잉' in product) : name = '떡볶이'
        
    elif '덮밥' in product : name = '덮밥'
    elif '순대' in product : name = '순대'
        
    elif '무침' in product : name = '무침'
    elif '감바스' in product : name = '감바스'
    elif ('해물' in product)or('씨푸드' in product)or('씨키트' in product)or('꽃게' in product)or('새우' in product): name = '씨푸드'
    elif ('쉬림프' in product)or('연어' in product)or('조개' in product)or('문어' in product): name = '씨푸드'

    elif ('콩불' in product) or ('불고기' in product) or ('비빔밥' in product) : name='한식'
    elif ('부침개' in product) or ('조림' in product) : name = '한식'

    elif '오리' in product : name = '오리'

    elif ('커리' in product) or ('카레' in product) : name = '카레'
    
    elif '샌드위치' in product : name = '샌드위치'
    elif ('볶음' in product) or ('잡채' in product) : name = '볶음'
    
    elif '쌈' in product : name = '쌈'
    elif ('국' in product) or ('육개장' in product) or ('파개장' in product) or ('수제비' in product) or ('칼제비' in product) : name = '국'
    elif '찜' in product : name = '찜'
    else: name='기타'
        
    return name

In [4]:
# def category_name22(product):
#     if '면'|'누들'|'짜장'|'짬뽕' in product : 
#         name = '전골'
#     else : 
#         name = '기타'
#     return name

In [21]:
#df['상세카테고리']=df['상품'].apply(lambda x : category_name22(x))
df['상세카테고리']=df['상품'].apply(lambda x : category_name(x))
df.to_csv('../../data/coupang_filter_detail.csv')
df['상세카테고리'].value_counts()

찌개/짜글이    38
전골        33
파스타       27
기타        21
씨푸드       20
해물탕       20
스테이크      18
생선탕       18
면         14
국수        13
감바스       13
닭         10
국         10
알탕        10
구이         9
한식         8
찜          7
무침         7
떡볶이        6
순대         4
마라탕        2
마파두부       2
쌈          2
샌드위치       2
볶음         2
족발         2
리조또        2
카레         2
오리         1
Name: 상세카테고리, dtype: int64

In [6]:
df.loc[df['카테고리명']==1].상세카테고리.value_counts()

찌개/짜글이    38
전골        23
생선탕       18
해물탕       17
국          9
알탕         8
구이         6
기타         5
씨푸드        3
순대         2
닭          2
한식         1
찜          1
오리         1
면          1
국수         1
Name: 상세카테고리, dtype: int64

In [8]:
df.loc[df['카테고리명']==2].상세카테고리.value_counts()

씨푸드    2
Name: 상세카테고리, dtype: int64

In [9]:
df.loc[df['카테고리명']==3].상세카테고리.value_counts()

스테이크    9
Name: 상세카테고리, dtype: int64

In [10]:
df.loc[df['카테고리명']==4].상세카테고리.value_counts()

파스타     27
감바스     10
국수       8
면        6
기타       4
스테이크     2
해물탕      2
전골       1
알탕       1
Name: 상세카테고리, dtype: int64

In [11]:
df.loc[(df['카테고리명']==4)&(df['상세카테고리']=='면')]

,Unnamed: 0,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리
164,167,4,6636134619,15151379986,82372899340,모두의맛집 샤오짠 탄탄면 (냉동),"모두의맛집 샤오짠 탄탄면 (냉동), 1개, 580g",8800,0,8800,"(100g당 1,517원)",4.0,8,0,"구성품(청경채, 돼지고기, 두반장 소스 등 함유), 생면으로 구성되어 있습니다.","청경채,돼지고기,생면",면
172,175,4,6867222379,16417092793,83608027927,마이셰프 소고기 야끼우동,"마이셰프 소고기 야끼우동, 1개, 451g",10900,0,10900,"(100g당 2,417원)",4.5,10,1,"소고기, 숙면, 숙주나물, 쪽파, 소스 등으로 구성되어 있습니다.","소고기,숙면,숙주나물,쪽파,소스",면
179,182,4,6775844398,15921388137,83128695588,명인만두 비빔만두쫄면 1인분 (냉동),"명인만두 비빔만두쫄면 1인분 (냉동), 2개, 370g",10200,0,10200,"(100g당 1,378원)",0.0,0,1,"쫄면, 만두, 소스로 구성되어 있습니다.","쫄면,만두,소스",면
200,203,4,6410059402,13743233723,80994055837,쉐푸드 통돈까스 김치 우동 밀키트 (냉동),"쉐푸드 통돈까스 김치 우동 밀키트 (냉동), 640g, 1개",14900,0,14900,"(100g당 2,328원)",4.5,34,1,"통돈까스, 볶음김치, 채소(양파, 대파, 청양고추), 팽이버섯, 우동면, 육수, 양...","볶음김치,채소,양파,대파,청양고추,팽이버섯,우동면,육수",면
203,206,4,6246947929,12641184013,79908701427,맛투데이 찐 해물 굴짬뽕탕 690g (냉동),"맛투데이 찐 해물 굴짬뽕탕 690g (냉동), 1세트",13810,0,13810,NaN,4.5,70,1,"해산물, 칼국수 면, 소스로 구성되어 있습니다.","해산물,칼국수면,소스",면
204,207,4,6246947928,12641184012,79908701416,맛투데이 찐 통 오징어 짬뽕탕 750g (냉동),"맛투데이 찐 통 오징어 짬뽕탕 750g (냉동), 1세트",17900,11%,15900,NaN,4.0,89,1,"해산물, 칼국수 면, 소스로 구성되어 있습니다.","해산물,칼국수면,소스",면


In [12]:
df.loc[df['카테고리명']==5].상세카테고리.value_counts()

떡볶이    2
순대     2
Name: 상세카테고리, dtype: int64

In [13]:
df.loc[df['카테고리명']==6].상세카테고리.value_counts()

기타      4
면       3
마라탕     2
마파두부    2
Name: 상세카테고리, dtype: int64

In [14]:
df.loc[df['카테고리명']==7].상세카테고리.value_counts()

씨푸드     15
전골       9
기타       8
닭        8
스테이크     7
무침       7
한식       7
찜        6
면        4
국수       4
떡볶이      4
감바스      3
구이       3
카레       2
리조또      2
족발       2
볶음       2
샌드위치     2
쌈        2
해물탕      1
알탕       1
국        1
Name: 상세카테고리, dtype: int64

In [16]:
df.loc[(df['카테고리명']==5)&(df['상세카테고리']=='마라샹궈')]

,Unnamed: 0,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리


In [33]:
df.loc[df['상품'].str.contains('잡체')]

,Unnamed: 0,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리


In [18]:
df.loc[df['카테고리명']==7].상품

223                     프레시지 더큰 블랙라벨 스테이크 세트
224       바다자리 국산깐새우 갈릭버터쉬림프 + 야채 씨키트 2인분 세트
225    바다자리 매콤한 순살아귀 불고기 + 야채 + 소스 씨키트 2~3인분
226      바다자리 자숙문어 피꼬막무침 씨키트 2~3인분 250g + 야채
227                            마이셰프 불고기 퀘사디아
                       ...                  
318                    쉐푸드 배추 만두 전골 밀키트 (냉동)
319                         마이셰프 알찬 소고기 스키야키
320                   바다자리 생새우 갈릭버터쉬림프 1~2인분
321                 바른식 등촌식 얼큰 미나리 샤브샤브 (냉장)
322                    바다자리 생새우 보일링쉬림프 3~4인분
Name: 상품, Length: 100, dtype: object

In [19]:
trend = {'감바스' : '해산물요리', '구이' : '새우구이, 생선구이, 조개구이' , '국' : '된장찌개·청국장, 국밥, 해장국, 국·탕, 순대국', '국수' : '국수·칼국수·수제비, 쌀국수', '닭' : '옻닭, 닭볶음탕, 백숙·토종닭, 닭갈비, 찜닭', '덮밥' : '덮밥',
 '떡볶이' : '떡볶이', '리조또' : '리소토·필라프' , '마라탕' : '마라탕, ', '마파두부' : '두부요리, 중식요리', '면' : '면류, 물냉면, 짜장면, 비빔냉면, 쫄면', '무침' :'채식·비건 요리', '볶음' : '볶음밥, 제육볶음·두루치기·불고기',
 '불고기' : '돼지갈비·주물럭·양념고기, 제육볶음·두루치기·불고기', '샌드위치' : '샌드위치', '생선탕' : '매운탕·해물탕, 생선찌개', '순대' : '순대', '스테이크' : '스테이크, 함박스테이크', '쌈' : '보쌈, 쌈밥', '씨푸드' : '해산물요리', '알탕' : '매운탕·해물탕, 국·탕', '오리' : '오리로스·주물럭·훈제', 
 '전골' : '나베·전골', '족발' : '족발', '찌개/짜글이' : '된장찌개·청국장, 고추장찌개·돼지찌개, 찌개, 생선찌개, 순두부찌개, 김치찌개·참치찌개, 부대찌개', '찜' : '갈비찜·뼈찜, 생선찜·해물찜, 찜닭', '카레' : '카레', '파스타' : '파스타',
 '한식' : '전·빈대떡·부침개, 비빔밥, 제육볶음·두루치기·불고기, 국수·칼국수·수제비', '해물탕' : '매운탕·해물탕, 해산물요리' }

In [24]:
df['상세카테고리'].unique()

array(['전골', '생선탕', '한식', '찌개/짜글이', '알탕', '해물탕', '기타', '국', '구이', '순대',
       '씨푸드', '닭', '찜', '오리', '면', '국수', '스테이크', '파스타', '감바스', '떡볶이',
       '마라탕', '마파두부', '무침', '쌈', '샌드위치', '볶음', '족발', '리조또', '카레'],
      dtype=object)

In [ ]:
dict1 = {'찌개/국' : '전골, 생선탕, 찌개/짜글이, 알탕, 해물탕, 국, 마라탕',
         '면/파스타' : '면, 국수, 파스타',
         '구이' : '구이, 스테이크',
         '볶음/튀김' : '볶음, 감바스, 순대',
         '조림/찜' : '찜, 마파두부, 족발',
         '전체' : '한식, 기타, 씨푸드, 닭, 오리, 떡볶이, 무침, 쌈, 샌드위치, 리조또, 카레'}

In [20]:
## 네이버 데이터랩 api 코드 이지만 쓸 일 없을 듯
# #-*- coding: utf-8 -*-
# import os
# import sys
# import urllib.request
# client_id = "YOUR_CLIENT_ID"
# client_secret = "YOUR_CLIENT_SECRET"
# url = "https://openapi.naver.com/v1/datalab/search";
# body = "{\"startDate\":\"2017-01-01\",\"endDate\":\"2017-04-30\",\"timeUnit\":\"month\",\"keywordGroups\":[{\"groupName\":\"한글\",\"keywords\":[\"한글\",\"korean\"]},{\"groupName\":\"영어\",\"keywords\":[\"영어\",\"english\"]}],\"device\":\"pc\",\"ages\":[\"1\",\"2\"],\"gender\":\"f\"}";

# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)
# request.add_header("Content-Type","application/json")
# response = urllib.request.urlopen(request, data=body.encode("utf-8"))
# rescode = response.getcode()
# if(rescode==200):
#     response_body = response.read()
#     print(response_body.decode('utf-8'))
# else:
#     print("Error Code:" + rescode)
